In [7]:
import cv2
import matplotlib.pyplot as plt
from datetime import datetime
import os
import numpy as np
import face_recognition

In [8]:
path = 'student_images'

In [9]:
images = []
classNames = []
mylist = os.listdir(path)
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

In [10]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)[0]
        encodeList.append(encoded_face)
    return encodeList
encoded_face_train = findEncodings(images)


In [11]:
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(', ')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            time = now.strftime('%I:%M:%S:%p')
            date = now.strftime('%d-%B-%Y')
            f.writelines(f'{name}, {time}, {date}\n')

In [12]:
faceCas = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(1)

if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError('Cannot open webcam')

while True:
    ret,frame = cap.read()
    cv2.imshow('Video', frame)

    faces = cv2.cvtColor(faces, cv2.COLOR_BGR2RGB)

    student = face_recognition.face_locations(faces)

    encoded_faces = face_recognition.face_encodings(faces, student)
    for encode_face, faceloc in zip(encoded_faces,student):
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
        matchIndex = np.argmin(faceDist)


        if matches[matchIndex]:
            name = classNames[matchIndex].upper().lower()
            
            x, y, w, h = faceloc
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)    

            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, name, (50,50), font, 3, (0, 0, 50), 2, cv2.LINE_4)
            cv2.imshow('Video', frame)

            markAttendance(name)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()